In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import string
import random
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from matplotlib import pyplot
from gensim.models import KeyedVectors

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ckkok\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [8]:
import os

In [5]:
# Reading the data
code_df = pd.read_csv("../../data/anime_codes.csv")

In [6]:
code_df.head()

,code,name,rating
0,5114,fullmetal_alchemist__brotherhood,9.22
1,11061,hunter_x_hunter_2011,9.12
2,28977,gintama°,9.11
3,9969,gintama,9.09
4,38524,shingeki_no_kyojin_season_3_part_2,9.07


In [23]:
# Read the review data
reviews_dir = "../../data/reviews"
all_reviews = list()

for index, review_doc in enumerate(os.listdir(reviews_dir)):
    current_dict = dict()
        
    # Get the current anime's code
    current_dict['code'] = review_doc.split('.')[0]
    
    # Get the current anime's reviews
    f = open(os.path.join(reviews_dir, review_doc), 'r', encoding="utf-8")
    current_dict['review'] = f.read()
    f.close()
    
    all_reviews.append(current_dict)

# Create a dataframe of the anime codes and their respective reviews
review_df = pd.DataFrame(all_reviews)
review_df.shape

(877, 2)

In [24]:
# Match the name and rating of animes in code_df to the anime reviews dataframe
review_df['code']=review_df['code'].astype(int)
df = pd.merge(review_df, code_df, on='code')
print(df.shape)
df.head()

(877, 4)


,code,review,name,rating
0,1,People who know me know that I'm not a fan of ...,cowboy_bebop,8.79
1,1000,There is a reason this is considered to be one...,uchuu_kaizoku_captain_herlock,7.72
2,1002,Many people will know the name of Hideaki Anno...,top_wo_nerae_2_diebuster,7.68
3,10033,"""Cooking is as masculine as judo, kickboxing o...",toriko,7.59
4,10049,After finishing the first series of Nurarihyon...,nurarihyon_no_mago__sennen_makyou,8.02


In [25]:
#Utitlity functions for removing ASCII characters, converting lower case, removing stop words, html and punctuation from description

def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)

def make_lower_case(text):
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text


In [26]:
df['cleaned'] = df['review'].apply(_removeNonAscii)
df['cleaned'] = df.cleaned.apply(func = make_lower_case)
df['cleaned'] = df.cleaned.apply(func = remove_stop_words)
df['cleaned'] = df.cleaned.apply(func=remove_punctuation)
df['cleaned'] = df.cleaned.apply(func=remove_html)
df.head()

,code,review,name,rating,cleaned
0,1,People who know me know that I'm not a fan of ...,cowboy_bebop,8.79,people know know i m fan episodic anime series...
1,1000,There is a reason this is considered to be one...,uchuu_kaizoku_captain_herlock,7.72,reason considered one greatest anime series ev...
2,1002,Many people will know the name of Hideaki Anno...,top_wo_nerae_2_diebuster,7.68,many people know name hideaki anno particular ...
3,10033,"""Cooking is as masculine as judo, kickboxing o...",toriko,7.59,cooking masculine judo kickboxing tae kwan do ...
4,10049,After finishing the first series of Nurarihyon...,nurarihyon_no_mago__sennen_makyou,8.02,finishing first series nurarihyon mago bit rel...
